# 1 
Из названия задания следует, что это тестовое задание и для DS, и для Дата Инженеров. При этом эта (первая часть задания) больше по ижненерной части, а построение модели различными способами и выбор лучшей - как раз DS часть. Я буду работать с CSV файлами, так как на текущий момент у меня нет практического опыта работы с SQL (я изучал данный язык и умею писать запросы, что готов продемонстрировать в дальнейшем, но при этом конкретно с серверами я не работал). Тем не менее, уже с сентября я планирую реализовывать проекты, работая напрямую с SQL

# 2
Начнем с использования классических методов машинного обучения в sklearn, а далее перейдем к PySpark. В качестве метрик сравнения моделей будем использовать Roc-Auc, а также Recall. В контексте нашей задачи важнее верно классифицировать опасные астероиды, нежели добиться максимальной общей точности. Поэтому Recall важнее, чем Precision и Accuracy (стремимся минимизировать ошибку второго рода)

### 2.1 Работа с Pandas, Sklearn

In [268]:
import pandas as pd
import numpy as np

In [269]:
df = pd.read_csv('neo.csv')
df

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,False,16.73,False
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,False,20.00,True
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,False,17.83,False
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,False,22.20,False
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,False,20.09,True
...,...,...,...,...,...,...,...,...,...,...
90831,3763337,(2016 VX1),0.026580,0.059435,52078.886692,1.230039e+07,Earth,False,25.00,False
90832,3837603,(2019 AD3),0.016771,0.037501,46114.605073,5.432121e+07,Earth,False,26.00,False
90833,54017201,(2020 JP3),0.031956,0.071456,7566.807732,2.840077e+07,Earth,False,24.60,False
90834,54115824,(2021 CN5),0.007321,0.016370,69199.154484,6.869206e+07,Earth,False,27.80,False


###### 2.1.1 Feature Engineering



Проверяем датафрейм на пустые значения для предоработки данных

In [270]:
print(df.isnull().sum())

id                    0
name                  0
est_diameter_min      0
est_diameter_max      0
relative_velocity     0
miss_distance         0
orbiting_body         0
sentry_object         0
absolute_magnitude    0
hazardous             0
dtype: int64


Пустых значений нет, и в целом датасет выглядит достаточно аккуратным, поэтому можно переходить к выбору признаков (выбросы удалим позже)

###### 2.1.2 Выбор признаков

In [271]:
df = df.drop(columns = ['id', 'name'])

In [272]:
print(df['sentry_object'].value_counts())
print('')
print(df['orbiting_body'].value_counts())

False    90836
Name: sentry_object, dtype: int64

Earth    90836
Name: orbiting_body, dtype: int64


In [273]:
print(df['hazardous'].value_counts())

False    81996
True      8840
Name: hazardous, dtype: int64


У нас нет значений null, а также есть две колонки с одним вариантов заполнения, поэтому их можно удалить (они нам не дают информацию об объекте)

In [274]:
df = df.drop(columns = ['orbiting_body', 'sentry_object'])
df

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,1.198271,2.679415,13569.249224,5.483974e+07,16.73,False
1,0.265800,0.594347,73588.726663,6.143813e+07,20.00,True
2,0.722030,1.614507,114258.692129,4.979872e+07,17.83,False
3,0.096506,0.215794,24764.303138,2.543497e+07,22.20,False
4,0.255009,0.570217,42737.733765,4.627557e+07,20.09,True
...,...,...,...,...,...,...
90831,0.026580,0.059435,52078.886692,1.230039e+07,25.00,False
90832,0.016771,0.037501,46114.605073,5.432121e+07,26.00,False
90833,0.031956,0.071456,7566.807732,2.840077e+07,24.60,False
90834,0.007321,0.016370,69199.154484,6.869206e+07,27.80,False


In [275]:
x = df.drop(columns = ['hazardous'])
for column in x:
    x[column] = (x[column] - x[column].min()) / (x[column].max() - x[column].min())
x

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude
0,0.031607,0.031607,0.056447,0.733141,0.312891
1,0.006999,0.006999,0.309922,0.821364,0.449312
2,0.019039,0.019039,0.481680,0.665740,0.358782
3,0.002531,0.002531,0.103726,0.339986,0.541093
4,0.006714,0.006714,0.179632,0.618634,0.453066
...,...,...,...,...,...
90831,0.000685,0.000685,0.219081,0.164371,0.657906
90832,0.000427,0.000427,0.193893,0.726208,0.699625
90833,0.000827,0.000827,0.031097,0.379640,0.641218
90834,0.000177,0.000177,0.291384,0.918352,0.774718


###### 2.1.3 Избавляемся от выбросов
Будем делать это классическим статистическим методом через перцентили. Так как датасет у нас достаточно большой, а признаков не так много, можем себе позволить удалить выбросы по каждом признаку отдельно

In [276]:

Q1 = np.percentile(df['est_diameter_min'], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(df['est_diameter_min'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
 
upper = np.where(df['est_diameter_min'] >= (Q3+1.5*IQR))

lower = np.where(df['est_diameter_min'] <= (Q1-1.5*IQR))
 

df.drop(upper[0], inplace = True)
df.drop(lower[0], inplace = True)
df.index = np.arange(0, len(df))

In [277]:
Q1 = np.percentile(df['est_diameter_max'], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(df['est_diameter_max'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
 
upper = np.where(df['est_diameter_max'] >= (Q3+1.5*IQR))

lower = np.where(df['est_diameter_max'] <= (Q1-1.5*IQR))
 

df.drop(upper[0], inplace = True)
df.drop(lower[0], inplace = True)
df.index = np.arange(0, len(df))


In [278]:
Q1 = np.percentile(df['relative_velocity'], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(df['relative_velocity'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
 
upper = np.where(df['relative_velocity'] >= (Q3+1.5*IQR))

lower = np.where(df['relative_velocity'] <= (Q1-1.5*IQR))
 

df.drop(upper[0], inplace = True)
df.drop(lower[0], inplace = True)
df.index = np.arange(0, len(df))


In [279]:
Q1 = np.percentile(df['miss_distance'], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(df['miss_distance'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
 
upper = np.where(df['miss_distance'] >= (Q3+1.5*IQR))

lower = np.where(df['miss_distance'] <= (Q1-1.5*IQR))
 

df.drop(upper[0], inplace = True)
df.drop(lower[0], inplace = True)
df.index = np.arange(0, len(df))


In [280]:
Q1 = np.percentile(df['absolute_magnitude'], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(df['absolute_magnitude'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
 
upper = np.where(df['absolute_magnitude'] >= (Q3+1.5*IQR))

lower = np.where(df['absolute_magnitude'] <= (Q1-1.5*IQR))
 

df.drop(upper[0], inplace = True)
df.drop(lower[0], inplace = True)
df.index = np.arange(0, len(df))
df

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,0.096506,0.215794,24764.303138,2.543497e+07,22.20,False
1,0.036354,0.081291,34297.587778,4.058569e+07,24.32,False
2,0.171615,0.383743,27529.472307,2.906912e+07,20.95,False
3,0.005328,0.011914,57544.470083,5.511502e+07,28.49,False
4,0.105817,0.236614,48425.840329,3.835526e+07,22.00,False
...,...,...,...,...,...,...
76662,0.026580,0.059435,52078.886692,1.230039e+07,25.00,False
76663,0.016771,0.037501,46114.605073,5.432121e+07,26.00,False
76664,0.031956,0.071456,7566.807732,2.840077e+07,24.60,False
76665,0.007321,0.016370,69199.154484,6.869206e+07,27.80,False


Теперь, когда мы избавились от выбросов, стоит перевести значения target колонки в числовые (1 для положительного результата, 0 - для отрицательного)

In [281]:
df.loc[df['hazardous'] == False, 'hazardous'] = 0
df.loc[df['hazardous'] == True, 'hazardous'] = 1
df

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,0.096506,0.215794,24764.303138,2.543497e+07,22.20,0
1,0.036354,0.081291,34297.587778,4.058569e+07,24.32,0
2,0.171615,0.383743,27529.472307,2.906912e+07,20.95,0
3,0.005328,0.011914,57544.470083,5.511502e+07,28.49,0
4,0.105817,0.236614,48425.840329,3.835526e+07,22.00,0
...,...,...,...,...,...,...
76662,0.026580,0.059435,52078.886692,1.230039e+07,25.00,0
76663,0.016771,0.037501,46114.605073,5.432121e+07,26.00,0
76664,0.031956,0.071456,7566.807732,2.840077e+07,24.60,0
76665,0.007321,0.016370,69199.154484,6.869206e+07,27.80,0


###### 2.1.4 Проверка датасета на сбалансированность

In [282]:
print(df['hazardous'].value_counts())

0    71633
1     5034
Name: hazardous, dtype: int64


Наш датасет очень несбалансирован, а также, избавившись от выбросов, мы избавились от почти половины опасных наблюдений. С выбросами работать не стоит, тем более у нас достаточно данных. Кажется хорошей идеей сделать датасет сбалансированным. В итоге мы будем работать примерно с 10 тысячами данных, чего обычно достаточно, чтобы обучить хорошую модель

In [283]:
from imblearn.under_sampling import RandomUnderSampler  

###### 2.1.5 Отделение признаков от target variable и нормализация данных

Обычно самый компромиссный и понятный вариант - перевод данных в диапазон от 0 до 1

In [284]:
x = df.drop(columns = ['hazardous'])
for column in x:
    x[column] = (x[column] - x[column].min()) / (x[column].max() - x[column].min())
x
y = df['hazardous']
y = y.astype('int')
under_sampler = RandomUnderSampler()
x, y = under_sampler.fit_resample(x, y)

In [285]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
x = x
y = y
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
y_train = y_train.astype('int')
y_test = y_test.astype('int')
pool_train = Pool(x_train, y_train)
pool_test = Pool(x_test)
model = CatBoostClassifier()
model.fit(pool_train)

Learning rate set to 0.025107
0:	learn: 0.6501941	total: 11ms	remaining: 11s
1:	learn: 0.6162303	total: 16ms	remaining: 7.99s
2:	learn: 0.5871639	total: 19.9ms	remaining: 6.62s
3:	learn: 0.5572173	total: 22.6ms	remaining: 5.63s
4:	learn: 0.5306629	total: 26.4ms	remaining: 5.25s
5:	learn: 0.5065583	total: 30.4ms	remaining: 5.03s
6:	learn: 0.4807237	total: 34.1ms	remaining: 4.84s
7:	learn: 0.4618543	total: 38.1ms	remaining: 4.73s
8:	learn: 0.4461934	total: 42ms	remaining: 4.62s
9:	learn: 0.4267787	total: 63.9ms	remaining: 6.33s
10:	learn: 0.4131149	total: 67.9ms	remaining: 6.1s
11:	learn: 0.3986504	total: 72ms	remaining: 5.92s
12:	learn: 0.3843477	total: 75.8ms	remaining: 5.76s
13:	learn: 0.3714433	total: 79.6ms	remaining: 5.61s
14:	learn: 0.3610486	total: 83.5ms	remaining: 5.48s
15:	learn: 0.3521834	total: 87.4ms	remaining: 5.37s
16:	learn: 0.3429963	total: 91.2ms	remaining: 5.27s
17:	learn: 0.3335148	total: 95ms	remaining: 5.18s
18:	learn: 0.3262502	total: 98.9ms	remaining: 5.11s
19:	l

In [286]:
print(roc_auc_score(y_test, model.predict_proba(x_test)[:, 1]))
print('')
print(classification_report(y_test, model.predict(x_test)))

0.9498837604003851

              precision    recall  f1-score   support

           0       0.98      0.86      0.92      1032
           1       0.87      0.99      0.93       982

    accuracy                           0.92      2014
   macro avg       0.93      0.92      0.92      2014
weighted avg       0.93      0.92      0.92      2014



Как мы видим, результаты достаточно хорошие

### 2.2 Делаем то же самое с помощью PySpark

Так как процедура аналогична предыдущей, я не буду делать множество комментариев



In [287]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [288]:
spark

In [289]:
df_pyspark = spark.read.option('header', 'true').csv('neo.csv', inferSchema = True)
df_pyspark.show()

+--------+-------------------+----------------+----------------+-----------------+--------------------+-------------+-------------+------------------+---------+
|      id|               name|est_diameter_min|est_diameter_max|relative_velocity|       miss_distance|orbiting_body|sentry_object|absolute_magnitude|hazardous|
+--------+-------------------+----------------+----------------+-----------------+--------------------+-------------+-------------+------------------+---------+
| 2162635|162635 (2000 SS164)|    1.1982708007|    2.6794149658| 13569.2492241812| 5.483974408284605E7|        Earth|        false|             16.73|    false|
| 2277475|  277475 (2005 WK4)|          0.2658|    0.5943468684| 73588.7266634981| 6.143812652395093E7|        Earth|        false|              20.0|     true|
| 2512244| 512244 (2015 YE18)|    0.7220295577|    1.6145071727|114258.6921290512| 4.979872494045679E7|        Earth|        false|             17.83|    false|
| 3596030|        (2012 BV13)|    

In [290]:
df_pyspark.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- est_diameter_min: double (nullable = true)
 |-- est_diameter_max: double (nullable = true)
 |-- relative_velocity: double (nullable = true)
 |-- miss_distance: double (nullable = true)
 |-- orbiting_body: string (nullable = true)
 |-- sentry_object: boolean (nullable = true)
 |-- absolute_magnitude: double (nullable = true)
 |-- hazardous: boolean (nullable = true)



In [291]:
df_pyspark = df_pyspark.select('est_diameter_min', 'est_diameter_max', 'relative_velocity', 'miss_distance', 'absolute_magnitude', 'hazardous')
import pyspark.sql.functions as F
from functools import reduce
cols = ['hazardous']
df_pyspark = reduce(lambda df_pyspark, c: df_pyspark.withColumn(c, F.when(df_pyspark[c] == False, 0).otherwise(1)), cols, df_pyspark)
df_pyspark.show()
df_pyspark.describe().show()
#train, test = df_pyspark.randomSplit([0.8, 0.2])

+----------------+----------------+-----------------+--------------------+------------------+---------+
|est_diameter_min|est_diameter_max|relative_velocity|       miss_distance|absolute_magnitude|hazardous|
+----------------+----------------+-----------------+--------------------+------------------+---------+
|    1.1982708007|    2.6794149658| 13569.2492241812| 5.483974408284605E7|             16.73|        0|
|          0.2658|    0.5943468684| 73588.7266634981| 6.143812652395093E7|              20.0|        1|
|    0.7220295577|    1.6145071727|114258.6921290512| 4.979872494045679E7|             17.83|        0|
|     0.096506147|    0.2157943048| 24764.3031380016| 2.543497272075825E7|              22.2|        0|
|    0.2550086879|    0.5702167609| 42737.7337647264| 4.627556700130072E7|             20.09|        1|
|    0.0363542322|    0.0812905344| 34297.5877783029| 4.058569122792288E7|             24.32|        0|
|    0.1716148941|    0.3837425691| 27529.4723069673| 2.90691214

Далее следует удаление выбросов, выделение признаков в одну колонку, а также нормализация данных. В PySpark это делается намного сложнее. Я воспользовался ресурсом https://medium.com/@connectwithghosh/basic-data-preparation-in-pyspark-capping-normalizing-and-scaling-252ee7acba7d. Теперь мы аппроксимируем 1-ый и 99-й перцентили в первой четверти таблицы, а затем заменяем все значения ниже 1-го перцентиля на 1-ый перцентиль, а все значения выше 99-го перцентиля на 99-й перцентиль. Данные нормализуются так же, как в прошлый раз, с помощью стандартных методов. Для того, чтобы модели МО заработали, нужно все признаки расположить в одной колонке, что мы и сделаем


In [292]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
# empty dictionary d
d = {}
# Fill in the entries one by one
for col in df_pyspark.columns[0:-1]:
      d[col] = df_pyspark.approxQuantile(col,[0.01,0.99],0.25)
      print(col+" done")

for col in df_pyspark.columns[0:-1]:
    df_new = df_pyspark.withColumn(col, \
    F.log(F.when(df_pyspark[col] < d[col][0],d[col][0])\
    .when(df_pyspark[col] > d[col][1], d[col][1])\
    .otherwise(df_pyspark[col] ) +1).alias(col))
    print(col+' done')
  
assembler = VectorAssembler().setInputCols\
            (df_new.columns[0:-1]).setOutputCol("features")
transformed = assembler.transform(df_new)
scaler = MinMaxScaler(inputCol="features",\
         outputCol="scaledFeatures")
scalerModel =  scaler.fit(transformed.select("features"))
scaledData = scalerModel.transform(transformed)
scaledData.show()
scaledData.describe().show()

est_diameter_min done
est_diameter_max done
relative_velocity done
miss_distance done
absolute_magnitude done
est_diameter_min done
est_diameter_max done
relative_velocity done
miss_distance done
absolute_magnitude done
+----------------+----------------+-----------------+--------------------+------------------+---------+--------------------+--------------------+
|est_diameter_min|est_diameter_max|relative_velocity|       miss_distance|absolute_magnitude|hazardous|            features|      scaledFeatures|
+----------------+----------------+-----------------+--------------------+------------------+---------+--------------------+--------------------+
|    1.1982708007|    2.6794149658| 13569.2492241812| 5.483974408284605E7|2.8752581200861167|        0|[1.1982708007,2.6...|[0.03160721509989...|
|          0.2658|    0.5943468684| 73588.7266634981| 6.143812652395093E7| 3.044522437723423|        1|[0.2658,0.5943468...|[0.00699859603558...|
|    0.7220295577|    1.6145071727|114258.69212905

In [293]:
df_pyspark = scaledData.select('scaledFeatures', 'hazardous')
df_pyspark = df_pyspark.withColumnRenamed("scaledFeatures","features") \
    .withColumnRenamed("hazardous","label")
df_pyspark.show(truncate=False)

+--------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                |label|
+--------------------------------------------------------------------------------------------------------+-----+
|[0.03160721509989549,0.03160721509926829,0.0564469971601135,0.7331408108414172,0.45565751533277815]     |0    |
|[0.006998596035584229,0.006998596034950869,0.309921777381082,0.8213640264416568,0.5959045684438167]     |1    |
|[0.019038843711854995,0.019038843710487936,0.4816795299939441,0.6657402134050745,0.505531713287505]     |0    |
|[0.0025308015102446153,0.0025308015082064154,0.10372604641935702,0.33998635113737546,0.6784546994462463]|0    |
|[0.006713805038940003,0.006713805037386336,0.17963159528942302,0.6186340740998456,0.5994479879455896]   |1    |
|[9.43346379005237E-4,9.433463777886626E-4,0.1439870971995387,0.5425579840014307,0.7509066354038

Далее мы решаем проблему несбалансированного датасета. Будем применять методику undersampling, так как датасет является большим и мы можем себе это позволить. Я воспользовался следующим ресурсом https://medium.com/@junwan01/oversampling-and-undersampling-with-pyspark-5dbc25cdf253

In [294]:

from pyspark.sql.functions import col, explode, array, lit

In [295]:
major_df = df_pyspark.filter(col("hazardous") == 0)
minor_df = df_pyspark.filter(col("hazardous") == 1)
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))

ratio: 9


In [296]:
sampled_majority_df = major_df.sample(False, 1/ratio)
combined_df_2 = sampled_majority_df.unionAll(minor_df)
df_pyspark = combined_df_2
df_pyspark.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.24537340422041...|    0|
|[0.00132054732545...|    0|
|[0.00520797616810...|    0|
|[4.92097937842928...|    0|
|[0.01115274228519...|    0|
|[0.01372457088350...|    0|
|[0.00290812969713...|    0|
|[7.89321714260847...|    0|
|[0.02310494977364...|    0|
|[0.00581847513135...|    0|
|[3.35496420608791...|    0|
|[0.01136038600160...|    0|
|[0.00210232550569...|    0|
|[0.00220216239511...|    0|
|[2.16207741782785...|    0|
|[0.00253080151024...|    0|
|[3.43685446388947...|    0|
|[2.16207741782785...|    0|
|[4.06605736290991...|    0|
|[4.32682930231217...|    0|
+--------------------+-----+
only showing top 20 rows



In [297]:
df_pyspark.groupby('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    0| 9041|
|    1| 8840|
+-----+-----+



In [298]:
train, test = df_pyspark.randomSplit([0.8, 0.2])

Теперь все готово для построения модели

In [299]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(featuresCol = 'features', labelCol = 'label', maxIter = 10)
model = gbt.fit(train)
predictions = model.transform(test)
predictions.select("prediction", "label", "features")
evaluator = BinaryClassificationEvaluator()
print('Test area under Roc', evaluator.evaluate(predictions))
print('')
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(predictions.count())
print('Accuracy : ', accuracy)

Test area under Roc 0.9161902803663541

Accuracy :  0.8752711496746204


К сожалению, для бинарной классификации не представлены функции, позволяющие считать Precision, Recall, и так далее. Тем не менее уже про AUC и Accuracy мы видим, что модель, построенная с помощью sklearn демонстрирует себя лучше. При этом использовались сильно разные способы обработки данных для этих двух вариантов, поэтому это могло повлиять на поведение моделей, тем более результаты различаются не очень сильно